# Машинное обучение, ФКН ВШЭ

## Практическое задание 8. Метод опорных векторов и аппроксимация ядер

### Общая информация
Дата выдачи: 05.02.2021

Мягкий дедлайн: 01:59MSK 21.02.2021

Жесткий дедлайн: 01:59MSK 24.02.2021

### Оценивание и штрафы
Каждая из задач имеет определенную «стоимость» (указана в скобках около задачи). Максимальная оценка за работу (без учёта бонусов) — 10 баллов.

Сдавать задание после указанного жёсткого срока сдачи нельзя. При выставлении неполного балла за задание в связи с наличием ошибок на усмотрение проверяющего предусмотрена возможность исправить работу на указанных в ответном письме условиях.

Задание выполняется самостоятельно. «Похожие» решения считаются плагиатом и все задействованные студенты (в том числе те, у кого списали) не могут получить за него больше 0 баллов (подробнее о плагиате см. на странице курса). Если вы нашли решение какого-то из заданий (или его часть) в открытом источнике, необходимо указать ссылку на этот источник в отдельном блоке в конце вашей работы (скорее всего вы будете не единственным, кто это нашел, поэтому чтобы исключить подозрение в плагиате, необходима ссылка на источник).

Неэффективная реализация кода может негативно отразиться на оценке.

### Формат сдачи
Задания сдаются через систему anytask. Посылка должна содержать:
* Ноутбук homework-practice-08-random-features-Username.ipynb

Username — ваша фамилия и имя на латинице именно в таком порядке

### О задании

На занятиях мы подробно обсуждали метод опорных векторов (SVM). В базовой версии в нём нет чего-то особенного — мы всего лишь используем специальную функцию потерь, которая не требует устремлять отступы к бесконечности; ей достаточно, чтобы отступы были не меньше +1. Затем мы узнали, что SVM можно переписать в двойственном виде, который, позволяет заменить скалярные произведения объектов на ядра. Это будет соответствовать построению модели в новом пространстве более высокой размерности, координаты которого представляют собой нелинейные модификации исходных признаков.

Ядровой SVM, к сожалению, довольно затратен по памяти (нужно хранить матрицу Грама размера $d \times d$) и по времени (нужно решать задачу условной оптимизации с квадратичной функцией, а это не очень быстро). Мы обсуждали, что есть способы посчитать новые признаки $\tilde \varphi(x)$ на основе исходных так, что скалярные произведения этих новых $\langle \tilde \varphi(x), \tilde \varphi(z) \rangle$ приближают ядро $K(x, z)$.

Мы будем исследовать аппроксимации методом Random Fourier Features (RFF, также в литературе встречается название Random Kitchen Sinks) для гауссовых ядер. Будем использовать формулы, которые немного отличаются от того, что было на лекциях (мы добавим сдвиги внутрь тригонометрических функций и будем использовать только косинусы, потому что с нужным сдвигом косинус превратится в синус):
$$\tilde \varphi(x) = (
\cos (w_1^T x + b_1),
\dots,
\cos (w_n^T x + b_n)
),$$
где $w_j \sim \mathcal{N}(0, 1/\sigma^2)$, $b_j \sim U[-\pi, \pi]$.

На новых признаках $\tilde \varphi(x)$ мы будем строить любую линейную модель.

Можно считать, что это некоторая новая парадигма построения сложных моделей. Можно направленно искать сложные нелинейные закономерности в данных с помощью градиентного бустинга или нейронных сетей, а можно просто нагенерировать большое количество случайных нелинейных признаков и надеяться, что быстрая и простая модель (то есть линейная) сможет показать на них хорошее качество. В этом задании мы изучим, насколько работоспособна такая идея.

### Алгоритм

Вам потребуется реализовать следующий алгоритм:
1. Понизить размерность выборки до new_dim с помощью метода главных компонент.
2. Для полученной выборки оценить гиперпараметр $\sigma^2$ с помощью эвристики (рекомендуем считать медиану не по всем парам объектов, а по случайному подмножеству из где-то миллиона пар объектов): $$\sigma^2 = \text{median}_{i, j = 1, \dots, \ell, i \neq j} \left\{\sum_{k = 1}^{d} (x_{ik} - x_{jk})^2 \right\}$$
3. Сгенерировать n_features наборов весов $w_j$ и сдвигов $b_j$.
4. Сформировать n_features новых признаков по формулам, приведённым выше.
5. Обучить линейную модель (логистическую регрессию или SVM) на новых признаках.
6. Повторить преобразования (PCA, формирование новых признаков) к тестовой выборке и применить модель.

Тестировать алгоритм мы будем на данных Fashion MNIST. Ниже код для их загрузки и подготовки.

In [1]:
import keras
from keras.datasets import fashion_mnist
(x_train_pics, y_train), (x_test_pics, y_test) = fashion_mnist.load_data()
x_train = x_train_pics.reshape(x_train_pics.shape[0], -1)
x_test = x_test_pics.reshape(x_test_pics.shape[0], -1)

__Задание 1. (5 баллов)__

Реализуйте алгоритм, описанный выше. Можете воспользоваться шаблоном класса ниже или написать свой интерфейс.

Ваша реализация должна поддерживать следующие опции:
1. Возможность задавать значения гиперпараметров new_dim (по умолчанию 50) и n_features (по умолчанию 1000).
2. Возможность включать или выключать предварительное понижение размерности с помощью метода главных компонент.
3. Возможность выбирать тип линейной модели (логистическая регрессия или SVM с линейным ядром).

Протестируйте на данных Fashion MNIST, сформированных кодом выше. Если на тесте у вас получилась доля верных ответов не ниже 0.84 с гиперпараметрами по умолчанию, то вы всё сделали правильно.

In [2]:
from sklearn.base import BaseEstimator, TransformerMixin
import numpy as np

class RFFPipeline(BaseEstimator, TransformerMixin):
    def __init__(self, n_features=1000, new_dim=50, use_PCA=True, classifier='logreg'):
        """        
        Implements pipeline, which consists of PCA decomposition,
        Random Fourier Features approximation and linear classification model.
        
        n_features, int: amount of synthetic random features generated with RFF approximation.

        new_dim, int: PCA output size.
        
        use_PCA, bool: whether to include PCA preprocessing.
        
        classifier, string: either 'svm' or 'logreg', a linear classification model to use on top of pipeline.
        
        Feel free to edit this template for your preferences.    
        """
        self.n_features = n_features
        self.use_PCA = use_PCA
        self.new_dim = new_dim
        self.classifier = classifier
        
        self.X = None
        self.y = None
        
        self.w = None
        self.b = None
        self.phi_wave = None
        
        self.model = None
        self.pca = None
        
    def fit(self, X, y):
        """
        Fit all parts of algorithm (PCA, RFF, Classification) to training set.
        """
        ## Реализация PCA
        if self.use_PCA == True:
            from sklearn.decomposition import PCA
            pca = PCA(n_components = self.new_dim)
            self.X = pca.fit_transform(X)
            self.pca = pca
        else:
            self.X = X
            
        self.y = y
        
        ## Оценка вариации
        
        ind11 = np.random.randint(0, self.X.shape[0], size = 1000000)
        ind21 = np.random.randint(0, self.X.shape[0], size = 1000000)
        
        ind12 = ind11[ind11!=ind21]
        ind22 = ind21[ind21!=ind11]
        
        sigma2 = np.median(np.sum((x_train[ind12] - x_train[ind22])**2, axis = 1))
        
        ## Генерация весов и сдвигов
        
        w = np.random.normal(0, scale = 1/np.sqrt(sigma2), size = (self.new_dim, self.n_features))
        b = np.random.uniform(-np.pi, np.pi, size = (1, self.n_features))
        
        ## Генерация признаков phi_wave
        
        phi_wave = np.cos(self.X@w + np.repeat(b, self.X.shape[0], axis = 0))
        
        ## Присвоение значений классу
        
        self.w = w
        self.b = b
        self.phi_wave = phi_wave
        
        ## Обучение
        
        if self.classifier == 'logreg':
            from sklearn.linear_model import LogisticRegression
            m = LogisticRegression()
            m.fit(self.phi_wave, self.y)
            self.model = m
        else:
            from sklearn.svm import SVC
            m = SVC(kernel = 'linear')
            m.fit(self.phi_wave, self.y)
            self.model = m
        
        return self
    
    def preparation(self, x_test):
        if self.use_PCA == True:
            from sklearn.decomposition import PCA
            x_test_new = self.pca.transform(x_test)
            x_test_new = np.cos(x_test_new@self.w + np.repeat(self.b, x_test_new.shape[0], axis = 0))
            return x_test_new
        else:
            return x_test

    def predict_proba(self, X):
        """
        Apply pipeline to obtain scores for input data.
        """
        if self.classifier == 'logreg':
            return self.model.predict_proba(self.preparation(X))
        else:
            return self.model.predict(self.preparation(X)) ## Подумать че б с этим сделать
            
        #raise NotImplementedError
        
    def predict(self, X):
        """
        Apply pipeline to obtain discrete predictions for input data.
        """
        return self.model.predict(self.preparation(X))
        #raise NotImplementedError

In [3]:
m = RFFPipeline()
m.fit(x_train, y_train)

RFFPipeline()

In [4]:
upca = True
new_dim = 50

if upca == True:
    preds = m.predict(x_test)

In [5]:
np.sum(preds==y_test)/preds.shape[0]

0.2443

__Задание 2. (3 балла)__

Сравните подход со случайными признаками с обучением SVM на исходных признаках. Попробуйте вариант с обычным (линейным) SVM и с ядровым SVM. Ядровой SVM может очень долго обучаться, поэтому можно делать любые разумные вещи для ускорения: брать подмножество объектов из обучающей выборки, например.

Сравните подход со случайными признаками с вариантом, в котором вы понижаете размерность с помощью PCA и обучаете градиентный бустинг. Используйте одну из реализаций CatBoost/LightGBM/XGBoost, не забудьте подобрать число деревьев и длину шага.

Сделайте выводы — насколько идея со случайными признаками работает? Сравните как с точки зрения качества, так и с точки зрения скорости обучения и применения.

In [ ]:
## Обучение SVM на случайных признаках (линейная)

In [ ]:
## Обучение SVM на исходных признаках (линейная)

In [ ]:
## Обучение пакетного SVM на случайных признаках (линейная)
from sklearn.svm import SVC
from datetime import datetime

## Возьмем параметры из первой задачи
new_dim = 50
n_features = 1000

## Сгенерируем новые признаки

pca = PCA(n_components = new_dim)
x_train_new = pca.fit_transform(x_train)
pca = PCA(n_components = new_dim)
x_test_new = pca.fit_transform(x_test)

w = np.random.normal(0, scale = 1/np.sqrt(sigma2), size = (self.new_dim, self.n_features))
b = np.random.uniform(-np.pi, np.pi, size = (1, self.n_features))

phi_wave_train = np.cos(x_train_new@w + np.repeat(b, x_train_new.shape[0], axis = 0))
phi_wave_test = np.cos(x_test_new@w + np.repeat(b, x_test_new.shape[0], axis = 0))

## Обучим на них модель

lsvc = SVC(kernel = 'linear')

now = datetime.now()

lsvc.fit(phi_wave_train, y_train)

t = datetime.now() - now

print(f'Было затрачено времени: {t}')

preds = lsvc.predict(phi_wave_test)

print(f'Accuracy score: {accuracy_score(preds, y_test)}')

In [ ]:
## Обучение пакетного SVM на исходных признаках (линейная), понижение размерности не требуется
from sklearn.svm import SVC
from datetime import datetime

## Возьмем параметры из первой задачи
new_dim = 50
n_features = 1000

## Обучаем на исходных признаках

lsvc = SVC(kernel = 'linear')

now = datetime.now()

lsvc.fit(x_train, y_train)

t = datetime.now() - now

print(f'Было затрачено времени: {t}')

preds = lsvc.predict(x_test)

print(f'Accuracy score: {accuracy_score(preds, y_test)}')

In [ ]:
## Обучение пакетного SVM на случайных признаках (ядровая)

In [ ]:
## Обучение пакетного SVM на исходных признаках (ядровая)

In [37]:
## Градиентный бустинг с понижением размерности

from catboost import CatBoostClassifier
import matplotlib.pyplot as plt
from datetime import datetime
from sklearn.metrics import accuracy_score
from sklearn.model_selection import GridSearchCV

pca = PCA(n_components = new_dim)
x_train_new = pca.fit_transform(x_train)
pca = PCA(n_components = new_dim)
x_test_new = pca.fit_transform(x_test)

parameters = {'iterations':[1, 10, 100, 150, 200], 'learning_rate':[0.01, 0.05, 0.3, 0.5, 0.6]}
cb_model = CatBoostClassifier()
gs = GridSearchCV(cb_model, parameters)
gs.fit(x_train_new, y_train)

0:	learn: 2.2668782	total: 140ms	remaining: 0us
0:	learn: 2.2673428	total: 105ms	remaining: 0us
0:	learn: 2.2665968	total: 136ms	remaining: 0us
0:	learn: 2.2665967	total: 123ms	remaining: 0us
0:	learn: 2.2673397	total: 154ms	remaining: 0us
0:	learn: 2.1279609	total: 101ms	remaining: 0us
0:	learn: 2.1302473	total: 151ms	remaining: 0us
0:	learn: 2.1265859	total: 114ms	remaining: 0us
0:	learn: 2.1265849	total: 138ms	remaining: 0us
0:	learn: 2.1302332	total: 140ms	remaining: 0us
0:	learn: 1.4787580	total: 95.4ms	remaining: 0us
0:	learn: 1.4920959	total: 112ms	remaining: 0us
0:	learn: 1.4728093	total: 129ms	remaining: 0us
0:	learn: 1.4725994	total: 149ms	remaining: 0us
0:	learn: 1.4921943	total: 130ms	remaining: 0us
0:	learn: 1.2903277	total: 131ms	remaining: 0us
0:	learn: 1.3099231	total: 121ms	remaining: 0us
0:	learn: 1.2840488	total: 121ms	remaining: 0us
0:	learn: 1.2836059	total: 112ms	remaining: 0us
0:	learn: 1.3101268	total: 130ms	remaining: 0us
0:	learn: 1.2757136	total: 117ms	remain

2:	learn: 1.0751425	total: 331ms	remaining: 773ms
3:	learn: 0.9509377	total: 439ms	remaining: 658ms
4:	learn: 0.8834174	total: 549ms	remaining: 549ms
5:	learn: 0.8186070	total: 658ms	remaining: 439ms
6:	learn: 0.7715343	total: 781ms	remaining: 335ms
7:	learn: 0.7273250	total: 888ms	remaining: 222ms
8:	learn: 0.6954521	total: 1.02s	remaining: 113ms
9:	learn: 0.6653953	total: 1.13s	remaining: 0us
0:	learn: 1.2903277	total: 115ms	remaining: 1.04s
1:	learn: 1.0412108	total: 215ms	remaining: 859ms
2:	learn: 0.8996052	total: 307ms	remaining: 717ms
3:	learn: 0.8193594	total: 419ms	remaining: 629ms
4:	learn: 0.7493637	total: 536ms	remaining: 536ms
5:	learn: 0.6997537	total: 637ms	remaining: 425ms
6:	learn: 0.6647208	total: 749ms	remaining: 321ms
7:	learn: 0.6342517	total: 851ms	remaining: 213ms
8:	learn: 0.6132469	total: 956ms	remaining: 106ms
9:	learn: 0.5913777	total: 1.07s	remaining: 0us
0:	learn: 1.3099231	total: 128ms	remaining: 1.15s
1:	learn: 1.0444655	total: 248ms	remaining: 992ms
2:	l

58:	learn: 1.3767270	total: 5.93s	remaining: 4.12s
59:	learn: 1.3684700	total: 6.04s	remaining: 4.02s
60:	learn: 1.3614767	total: 6.14s	remaining: 3.93s
61:	learn: 1.3541651	total: 6.25s	remaining: 3.83s
62:	learn: 1.3468081	total: 6.36s	remaining: 3.73s
63:	learn: 1.3387772	total: 6.46s	remaining: 3.63s
64:	learn: 1.3316295	total: 6.57s	remaining: 3.54s
65:	learn: 1.3241806	total: 6.69s	remaining: 3.45s
66:	learn: 1.3180734	total: 6.79s	remaining: 3.35s
67:	learn: 1.3100639	total: 6.9s	remaining: 3.25s
68:	learn: 1.3040232	total: 7s	remaining: 3.14s
69:	learn: 1.2970958	total: 7.11s	remaining: 3.04s
70:	learn: 1.2900092	total: 7.21s	remaining: 2.94s
71:	learn: 1.2834851	total: 7.32s	remaining: 2.85s
72:	learn: 1.2772447	total: 7.43s	remaining: 2.75s
73:	learn: 1.2705117	total: 7.51s	remaining: 2.64s
74:	learn: 1.2636084	total: 7.61s	remaining: 2.54s
75:	learn: 1.2577286	total: 7.73s	remaining: 2.44s
76:	learn: 1.2519009	total: 7.82s	remaining: 2.33s
77:	learn: 1.2454947	total: 7.92s	r

20:	learn: 1.7972244	total: 2.26s	remaining: 8.49s
21:	learn: 1.7813293	total: 2.35s	remaining: 8.32s
22:	learn: 1.7652535	total: 2.44s	remaining: 8.19s
23:	learn: 1.7493937	total: 2.54s	remaining: 8.04s
24:	learn: 1.7347550	total: 2.64s	remaining: 7.93s
25:	learn: 1.7202504	total: 2.75s	remaining: 7.81s
26:	learn: 1.7062386	total: 2.84s	remaining: 7.68s
27:	learn: 1.6914624	total: 2.95s	remaining: 7.59s
28:	learn: 1.6767196	total: 3.06s	remaining: 7.48s
29:	learn: 1.6639507	total: 3.16s	remaining: 7.37s
30:	learn: 1.6518998	total: 3.26s	remaining: 7.27s
31:	learn: 1.6393726	total: 3.36s	remaining: 7.13s
32:	learn: 1.6269839	total: 3.46s	remaining: 7.02s
33:	learn: 1.6153375	total: 3.57s	remaining: 6.93s
34:	learn: 1.6037846	total: 3.67s	remaining: 6.82s
35:	learn: 1.5931582	total: 3.76s	remaining: 6.68s
36:	learn: 1.5815469	total: 3.86s	remaining: 6.57s
37:	learn: 1.5706930	total: 3.98s	remaining: 6.49s
38:	learn: 1.5601546	total: 4.07s	remaining: 6.37s
39:	learn: 1.5497590	total: 4.1

84:	learn: 1.2001157	total: 8.95s	remaining: 1.58s
85:	learn: 1.1951059	total: 9.05s	remaining: 1.47s
86:	learn: 1.1902028	total: 9.14s	remaining: 1.36s
87:	learn: 1.1846048	total: 9.26s	remaining: 1.26s
88:	learn: 1.1792625	total: 9.35s	remaining: 1.16s
89:	learn: 1.1738282	total: 9.45s	remaining: 1.05s
90:	learn: 1.1683745	total: 9.55s	remaining: 945ms
91:	learn: 1.1628984	total: 9.65s	remaining: 839ms
92:	learn: 1.1578317	total: 9.75s	remaining: 734ms
93:	learn: 1.1526049	total: 9.85s	remaining: 629ms
94:	learn: 1.1474178	total: 9.94s	remaining: 523ms
95:	learn: 1.1429180	total: 10s	remaining: 418ms
96:	learn: 1.1376805	total: 10.1s	remaining: 313ms
97:	learn: 1.1329186	total: 10.2s	remaining: 209ms
98:	learn: 1.1283564	total: 10.3s	remaining: 104ms
99:	learn: 1.1236646	total: 10.4s	remaining: 0us
0:	learn: 2.2673397	total: 122ms	remaining: 12.1s
1:	learn: 2.2333478	total: 238ms	remaining: 11.6s
2:	learn: 2.2009542	total: 358ms	remaining: 11.6s
3:	learn: 2.1694079	total: 475ms	remai

...

65:	learn: 0.3185698	total: 6.42s	remaining: 13s
66:	learn: 0.3175845	total: 6.5s	remaining: 12.9s
67:	learn: 0.3161245	total: 6.6s	remaining: 12.8s
68:	learn: 0.3148550	total: 6.69s	remaining: 12.7s
69:	learn: 0.3131305	total: 6.76s	remaining: 12.6s
70:	learn: 0.3118375	total: 6.86s	remaining: 12.5s
71:	learn: 0.3100679	total: 6.95s	remaining: 12.4s
72:	learn: 0.3089889	total: 7.03s	remaining: 12.2s
73:	learn: 0.3050447	total: 7.13s	remaining: 12.1s
74:	learn: 0.3026334	total: 7.22s	remaining: 12s
75:	learn: 0.3018590	total: 7.31s	remaining: 11.9s
76:	learn: 0.3003856	total: 7.39s	remaining: 11.8s
77:	learn: 0.2994533	total: 7.5s	remaining: 11.7s
78:	learn: 0.2979150	total: 7.59s	remaining: 11.6s
79:	learn: 0.2967529	total: 7.7s	remaining: 11.5s
80:	learn: 0.2958702	total: 7.8s	remaining: 11.5s
81:	learn: 0.2942655	total: 7.89s	remaining: 11.4s
82:	learn: 0.2930574	total: 7.99s	remaining: 11.3s
83:	learn: 0.2914940	total: 8.09s	remaining: 11.2s
84:	learn: 0.2906842	total: 8.2s	remaini

26:	learn: 0.4224999	total: 2.67s	remaining: 17.1s
27:	learn: 0.4155292	total: 2.77s	remaining: 17s
28:	learn: 0.4129389	total: 2.88s	remaining: 17s
29:	learn: 0.4083073	total: 2.97s	remaining: 16.9s
30:	learn: 0.4061568	total: 3.09s	remaining: 16.8s
31:	learn: 0.4017323	total: 3.19s	remaining: 16.8s
32:	learn: 0.3987226	total: 3.29s	remaining: 16.6s
33:	learn: 0.3942742	total: 3.41s	remaining: 16.7s
34:	learn: 0.3906384	total: 3.5s	remaining: 16.5s
35:	learn: 0.3881542	total: 3.6s	remaining: 16.4s
36:	learn: 0.3847926	total: 3.7s	remaining: 16.3s
37:	learn: 0.3791625	total: 3.81s	remaining: 16.2s
38:	learn: 0.3771650	total: 3.9s	remaining: 16.1s
39:	learn: 0.3747767	total: 4.01s	remaining: 16s
40:	learn: 0.3709320	total: 4.11s	remaining: 15.9s
41:	learn: 0.3669440	total: 4.2s	remaining: 15.8s
42:	learn: 0.3650992	total: 4.31s	remaining: 15.7s
43:	learn: 0.3619848	total: 4.41s	remaining: 15.6s
44:	learn: 0.3596011	total: 4.51s	remaining: 15.5s
45:	learn: 0.3578074	total: 4.59s	remainin

187:	learn: 0.1863044	total: 18.2s	remaining: 1.16s
188:	learn: 0.1853302	total: 18.3s	remaining: 1.06s
189:	learn: 0.1848347	total: 18.4s	remaining: 967ms
190:	learn: 0.1838086	total: 18.5s	remaining: 870ms
191:	learn: 0.1823027	total: 18.6s	remaining: 774ms
192:	learn: 0.1819335	total: 18.7s	remaining: 677ms
193:	learn: 0.1813189	total: 18.8s	remaining: 580ms
194:	learn: 0.1801141	total: 18.9s	remaining: 484ms
195:	learn: 0.1793307	total: 19s	remaining: 387ms
196:	learn: 0.1787146	total: 19.1s	remaining: 291ms
197:	learn: 0.1782829	total: 19.2s	remaining: 194ms
198:	learn: 0.1773741	total: 19.3s	remaining: 97.2ms
199:	learn: 0.1766224	total: 19.5s	remaining: 0us
0:	learn: 1.4721816	total: 140ms	remaining: 27.8s
1:	learn: 1.2222294	total: 256ms	remaining: 25.4s
2:	learn: 1.0607489	total: 405ms	remaining: 26.6s
3:	learn: 0.9607443	total: 535ms	remaining: 26.2s
4:	learn: 0.8770341	total: 655ms	remaining: 25.5s
5:	learn: 0.8196709	total: 777ms	remaining: 25.1s
6:	learn: 0.7740150	total: 

148:	learn: 0.2774749	total: 17s	remaining: 5.81s
149:	learn: 0.2768704	total: 17.1s	remaining: 5.69s
150:	learn: 0.2766050	total: 17.2s	remaining: 5.58s
151:	learn: 0.2760843	total: 17.3s	remaining: 5.46s
152:	learn: 0.2745703	total: 17.4s	remaining: 5.34s
153:	learn: 0.2736445	total: 17.5s	remaining: 5.22s
154:	learn: 0.2733021	total: 17.6s	remaining: 5.11s
155:	learn: 0.2729422	total: 17.7s	remaining: 5s
156:	learn: 0.2719330	total: 17.9s	remaining: 4.89s
157:	learn: 0.2714691	total: 17.9s	remaining: 4.77s
158:	learn: 0.2708123	total: 18.1s	remaining: 4.66s
159:	learn: 0.2704689	total: 18.2s	remaining: 4.54s
160:	learn: 0.2697008	total: 18.3s	remaining: 4.42s
161:	learn: 0.2692751	total: 18.4s	remaining: 4.31s
162:	learn: 0.2690566	total: 18.5s	remaining: 4.2s
163:	learn: 0.2686553	total: 18.6s	remaining: 4.08s
164:	learn: 0.2681892	total: 18.7s	remaining: 3.97s
165:	learn: 0.2677260	total: 18.8s	remaining: 3.85s
166:	learn: 0.2669397	total: 18.9s	remaining: 3.74s
167:	learn: 0.2664

GridSearchCV(estimator=<catboost.core.CatBoostClassifier object at 0x7f9d2af092e8>,
             param_grid={'iterations': [1, 10, 100, 150, 200],
                         'learning_rate': [0.01, 0.05, 0.3, 0.5, 0.6]})

__Задание 3. (2 балла)__

Проведите эксперименты:
1. Помогает ли предварительное понижение размерности с помощью PCA? 
2. Как зависит итоговое качество от n_features? Выходит ли оно на плато при росте n_features?
3. Важно ли, какую модель обучать — логистическую регрессию или SVM?

### Обучение линейной SVM модели с PCA и без

### Обучение логистической модели с PCA и без

### Бонус

__Задание 4. (Максимум 2 балла)__

Как вы, должно быть, помните с курса МО-1, многие алгоритмы машинного обучения работают лучше, если признаки данных некоррелированы. Оказывается, что для RFF существует модификация, позволяющая получать ортогональные случайные признаки (Orthogonal Random Features, ORF). Об этом методе можно прочитать в [статье](https://proceedings.neurips.cc/paper/2016/file/53adaf494dc89ef7196d73636eb2451b-Paper.pdf). Реализуйте класс для вычисления ORF по аналогии с основным заданием. Обратите внимание, что ваш класс должен уметь работать со случаем n_features > new_dim (в статье есть замечание на этот счет). Проведите эксперименты, сравнивающие RFF и ORF, сделайте выводы.

In [ ]:
# Your code here: (￣▽￣)/♫•*¨*•.¸¸♪

__Задание 5. (Максимум 2 балла)__

Поэкспериментируйте с функциями для вычисления новых случайных признаков. Не обязательно использовать косинус от скалярного произведения — можно брать знак от него, хэш и т.д. Придумайте побольше вариантов для генерации признаков и проверьте, не получается ли с их помощью добиваться более высокого качества. Также можете попробовать другой классификатор поверх случайных признаков, сравните результаты.

In [ ]:
# Your code here: (￣▽￣)/♫•*¨*•.¸¸♪